In [235]:
import pandas as pd
import numpy as np
from datetime import date
import xgboost as xgb

In [236]:
#load data
def load_data(file_name):
    records = list()
    df = pd.read_csv(file_name)
    for idx, row in df.iterrows():
        index, date, speed = row["id"], row["date"], row["speed"]
        records.append((index, date, speed))

    return records
                       
def load_data_test(file_name):
    records = list()
    df = pd.read_csv(file_name)
    for idx, row in df.iterrows():
        index, date = row["id"], row["date"]
        records.append((index, date))

    return records                      



In [237]:
train_file = 'train.csv'
dtrain = load_data(train_file)
test_file = 'test.csv'
dtest = load_data_test(test_file)
print(dtrain[0][1])
print(dtest[0])

1/1/2017 0:00
(0, '1/1/2018 2:00')


In [238]:
#extracte date into year+month+day,int(month),int(hour), weekday(1-7)
def extract_date(s, test=False):
    dates,time = s[1].split(' ')
    day, month, year = dates.split('/')
    hour, minute = time.split(':')
    weekday = date(int(year),int(month),int(day)).weekday()+1
    if int(day)<10:
        day = '0'+day
    if int(month)<10:
        month = '0'+month
    if test == True:
        return (s[0], year+month+day,int(month),int(hour), weekday)
    if test == False:
        return (s[0], year+month+day,int(month),int(hour), weekday, s[2])

    


In [239]:
# dataset and dates are list of tuples, but not dataframe, later they will turn back to panda dataframes 
train_date = list()
for entry in dtrain:
    entry = extract_date(entry)
    train_date.append(entry)
    
test_date = list()
for entry in dtest:
    entry = extract_date(entry, True)
    test_date.append(entry)

print(dtrain[0])
print(dtest[0])
print(train_date[0])
print(test_date[0])
print(len(dtrain), len(train_date), len(dtest), len(test_date))

(0, '1/1/2017 0:00', 43.00293016)
(0, '1/1/2018 2:00')
(0, '20170101', 1, 0, 7, 43.00293016)
(0, '20180101', 1, 2, 1)
14006 14006 3504 3504


In [240]:
train_df =pd.DataFrame(train_date)
train_df.columns = ['id','date_id', 'month', 'hour', 'weekdays', 'speed']
print(train_df)
test_df =pd.DataFrame(test_date)
test_df.columns = ['id','date_id', 'month', 'hour', 'weekdays']

          id   date_id  month  hour  weekdays      speed
0          0  20170101      1     0         7  43.002930
1          1  20170101      1     1         7  46.118696
2          2  20170101      1     2         7  44.294158
3          3  20170101      1     3         7  41.067468
4          4  20170101      1     4         7  46.448653
...      ...       ...    ...   ...       ...        ...
14001  14001  20181231     12    12         1  19.865269
14002  14002  20181231     12    15         1  17.820375
14003  14003  20181231     12    16         1  12.501851
14004  14004  20181231     12    18         1  15.979319
14005  14005  20181231     12    20         1  40.594183

[14006 rows x 6 columns]


In [241]:
temp_date = train_df.loc[0,'date_id']
print(temp_date)

20170101


In [242]:
#get weekends
def weekends(s):
    if s in [1,2,3,4,5]:
        return 0
    if s in [6,7]:
        return 1

# get rushing hours
def rushing_hours(s):
    #6-21 is the hour after rushing hour
    if s in [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]:
        return 1
    else:
        return 0 

    
 # Hong Kong public holidays    
holiday_list = ['02012017', '28012017', '30012017', '31012017','04042017','14042017',
           '15042017','17042017','01052017','03052017','30052017','01072017',
           '02102017','05102017','28102017','25122017','26122017',
           '01012018','16022018','17022018','19022018','30032018','31032018',
           '02042018','05042018','01052018','22052018','18062018','02072018',
           '25092018','01102018','17102018','25122018','26122018']

#get holidays
def holidays(s):
    if s in holiday_list:
        return 1
    else:
        return 0

#get max min mean speed of that day
def max_min_mean_speed(df,df2):
    #df is train df, df2 is test df
    #calculate max min mean speed for a day    
    temp_date = df.loc[0,'date_id']
    full_speed_list = []
    temp_speed_list = []
    for index in range(len(df)):
        if df.loc[index, 'date_id'] == temp_date:
            temp_speed_list.append(df.loc[index,'speed'])
        else:
            full_speed_list.append([temp_date, np.max(temp_speed_list),np.min(temp_speed_list),np.mean(temp_speed_list)])
            temp_date = df.loc[index,'date_id']
            temp_speed_list = []
            temp_speed_list.append(df.loc[index,'speed'])
    full_speed_list.append([temp_date,np.max(temp_speed_list),np.min(temp_speed_list),np.mean(temp_speed_list)])
    #print(temp_speed_list)
    #print(full_speed_list)
    #print(np.size(full_speed_list,axis = 0))
    temp_date = df.loc[0,'date_id']
    temp_speed_list_index = 0
    for index in range(len(df)):
        if df.loc[index, 'date_id'] == temp_date:
            #print(index)
            #print(temp_speed_list_index)
            df.loc[index,'max_speed']=full_speed_list[temp_speed_list_index][1]
            df.loc[index,'min_speed']=full_speed_list[temp_speed_list_index][2]
            df.loc[index,'mean_speed']=full_speed_list[temp_speed_list_index][3]
            #print(index)
        else:
            temp_date = df.loc[index, 'date_id']
            temp_speed_list_index = temp_speed_list_index+1
            df.loc[index,'max_speed']=full_speed_list[temp_speed_list_index][1]
            df.loc[index,'min_speed']=full_speed_list[temp_speed_list_index][2]
            df.loc[index,'mean_speed']=full_speed_list[temp_speed_list_index][3]
    #temp_date = df2.loc[0,'date_id']
    
    #transfer max min mean speed to test df
    temp_speed_list_index = 0
    for index in range(len(df2)):
        while (full_speed_list[temp_speed_list_index][0] != df2.loc[index, 'date_id']):
            temp_speed_list_index=temp_speed_list_index+1
        if df2.loc[index, 'date_id'] == full_speed_list[temp_speed_list_index][0]:
            #print(index)
            #print(temp_speed_list_index)
            df2.loc[index,'max_speed']=full_speed_list[temp_speed_list_index][1]
            df2.loc[index,'min_speed']=full_speed_list[temp_speed_list_index][2]
            df2.loc[index,'mean_speed']=full_speed_list[temp_speed_list_index][3]
            #print(index)

    
    return df,df2



In [243]:
train_df['weekends'] = train_df.weekdays.apply(weekends)
train_df['holiday'] = train_df.apply(lambda x: 1 if x['weekends']==1 else holidays(x['date_id']), axis=1)
#train_df= train_df.drop(['weekdays','weekends'],axis=1)

print(train_df.columns) 
#rushing_hours
train_df['rushing_hour'] = train_df.hour.apply(rushing_hours)
#weekend no rushing/ commented it is related
#train_df['rushing_hour'] = train_df.['hour'].apply(lambda x: 0 if x['weekends']==1 else x['rushing_hour'], axis=1)
train_df= train_df.drop(['weekends'],axis=1)

test_df['weekends'] = test_df.weekdays.apply(weekends)
# weekend and pulic holidays are all holidays, so drop weekend later
test_df['holiday'] = test_df.apply(lambda x: 1 if x['weekends']==1 else holidays(x['date_id']), axis=1)
test_df['rushing_hour'] = test_df.hour.apply(rushing_hours)
#weekend no rushing/ commented it is related
#test_df['rushing_hour'] = test_df.apply(lambda x: 0 if x['weekends']==1 else x['rushing_hour'], axis=1)
#rushing_hours
#test_df =test_df.drop(['weekdays','weekends'],axis=1)
test_df= test_df.drop(['weekends'],axis=1)

train_df['max_speed'] = 0
train_df['min_speed'] = 0
train_df['mean_speed'] = 0

test_df['max_speed'] = 0
test_df['min_speed'] = 0
test_df['mean_speed'] = 0

train_df,test_df = max_min_mean_speed(train_df,test_df)
#train_df= train_df.drop(['max_speed','min_speed'],axis=1)
#test_df= test_df.drop(['max_speed','min_speed'],axis=1)
#test_df = max_min_mean_speed(test_df)
#print(train_df[5:15])
#print(train_df[13925:13926]) #chrismas
print(train_df)
print(test_df)

Index(['id', 'date_id', 'month', 'hour', 'weekdays', 'speed', 'weekends',
       'holiday'],
      dtype='object')
          id   date_id  month  hour  weekdays      speed  holiday  \
0          0  20170101      1     0         7  43.002930        1   
1          1  20170101      1     1         7  46.118696        1   
2          2  20170101      1     2         7  44.294158        1   
3          3  20170101      1     3         7  41.067468        1   
4          4  20170101      1     4         7  46.448653        1   
...      ...       ...    ...   ...       ...        ...      ...   
14001  14001  20181231     12    12         1  19.865269        0   
14002  14002  20181231     12    15         1  17.820375        0   
14003  14003  20181231     12    16         1  12.501851        0   
14004  14004  20181231     12    18         1  15.979319        0   
14005  14005  20181231     12    20         1  40.594183        0   

       rushing_hour  max_speed  min_speed  mean_speed  


In [244]:
#get the speed from last section(hour), if there is no last section, get the last section of the last section. 
#Until no record, then it is 0
def get_last_speed_for_train(df):
    for current_index in range(len(df)):
        last_index = current_index-1
        while last_index>=0 and df.loc[last_index,'speed']==0:
            last_index = last_index-1
        if last_index<0:
            df.loc[current_index,'last_speed'] = df.loc[current_index,'speed']
        else:
            df.loc[current_index,'last_speed'] = df.loc[last_index,'speed']

#get the speed from next section(hour), if there is no last section, get the next section of the next section. 
#Until no record, then it is 0
def get_next_speed_for_train(df):
    for current_index in range(len(df)):
        next_index = current_index+1
        while next_index<len(df) and df.loc[next_index,'speed']==0:
            next_index = next_index+1
        if next_index==len(df):
            df.loc[current_index,'next_speed'] = df.loc[current_index,'speed']
        else:
            df.loc[current_index,'next_speed'] = df.loc[next_index,'speed']

#turn interger hour into string, and 0 prefix if it is less than 10            
def hours_to_string(s):
    if s<10:
        s = '0'+str(s)
        return s
    else:
        return str(s)

#close speeds are the last section speed and next section speed
#get last section speed and next section speed, temp_df is the full list the is concatenated with train df and test df.
#Test data still have no close speed after this function. 
#Close speeds of Test data are store in temp_df and needed to be extract in next function.
def get_close_speed_for_test_train(train_df,test_df):
    speed_df = train_df.loc[:, ['date_id','hour','speed']].copy()
    speed_df['str_hour']=speed_df['hour'].apply(hours_to_string)

    speed_df['sort_id']=speed_df['date_id']+speed_df['str_hour']
    speed_df = speed_df.drop(['date_id','hour','str_hour'],axis=1)
    #print(speed_df)
    test_df['str_hour']=test_df['hour'].apply(hours_to_string)
    test_df['sort_id']=test_df['date_id']+test_df['str_hour']
    test_df = test_df.drop(['str_hour'],axis=1)

    temp_test_df = test_df.copy()

    temp_test_df = temp_test_df[['sort_id']]
    temp_test_df['speed']=0
    #print(temp_test_df)

    #speed_df = speed_df.drop(['hour','str_hour'],axis=1)
    #temp_df = temp_test_df.join(speed_df, how='outer')
    temp_df = pd.concat([speed_df,temp_test_df], ignore_index=True) 
    temp_df = temp_df.sort_values('sort_id',ascending=True, ignore_index=True)
    temp_df['last_speed'] = 0
    temp_df['next_speed'] = 0
    #print(temp_df)
    get_last_speed_for_train(train_df)
    get_next_speed_for_train(train_df)
    get_last_speed_for_train(temp_df)
    get_next_speed_for_train(temp_df)
    print(temp_df)
    return temp_df,train_df,test_df

#make the speed from temp_df(actually it is the df with train and test record, extract the close speeds to test df)    
def transfer_close_speed_to_test(temp_df, test_df):
    temp_index = 0
    temp_len = len(temp_df)
    for i in range(len(test_df)):
        while temp_df.loc[temp_index,'sort_id']!=test_df.loc[i,'sort_id'] and temp_index<temp_len:
            temp_index = temp_index+1
        if temp_index<temp_len:
            test_df.loc[i,'last_speed'] = temp_df.loc[temp_index,'last_speed']
            test_df.loc[i,'next_speed'] = temp_df.loc[temp_index,'next_speed']
        else:
            test_df.loc[i,'last_speed']= 0
            test_df.loc[i,'next_speed'] = 0

In [245]:
# generate the speed from last hour and next hour
train_df['last_speed'] = 0
train_df['next_speed'] = 0
test_df['last_speed'] = 0
test_df['next_speed'] = 0
temp_df,train_df,test_df = get_close_speed_for_test(train_df,test_df)
transfer_close_speed_to_test(temp_df, test_df)
print(test_df)
print(train_df)

           speed     sort_id  last_speed  next_speed
0      43.002930  2017010100   43.002930   46.118696
1      46.118696  2017010101   43.002930   44.294158
2      44.294158  2017010102   46.118696   41.067468
3      41.067468  2017010103   44.294158   46.448653
4      46.448653  2017010104   41.067468   46.797766
...          ...         ...         ...         ...
17505   0.000000  2018123119   15.979319   40.594183
17506  40.594183  2018123120   15.979319   40.594183
17507   0.000000  2018123121   40.594183    0.000000
17508   0.000000  2018123122   40.594183    0.000000
17509   0.000000  2018123123   40.594183    0.000000

[17510 rows x 4 columns]
        id   date_id  month  hour  weekdays  holiday  rushing_hour  max_speed  \
0        0  20180101      1     2         1        0             0  50.626977   
1        1  20180101      1     5         1        0             0  50.626977   
2        2  20180101      1     7         1        0             1  50.626977   
3        3  20

In [246]:
#seperate to training and validation
train_s, validation = np.split(train_df, [int(.9*len(train_df))])
#validation, train_s = np.split(train_df, [int(.3*len(train_df))])
print(train_s)
print(validation)

          id   date_id  month  hour  weekdays      speed  holiday  \
0          0  20170101      1     0         7  43.002930        1   
1          1  20170101      1     1         7  46.118696        1   
2          2  20170101      1     2         7  44.294158        1   
3          3  20170101      1     3         7  41.067468        1   
4          4  20170101      1     4         7  46.448653        1   
...      ...       ...    ...   ...       ...        ...      ...   
12600  12600  20180919      9    16         3   9.875843        0   
12601  12601  20180919      9    19         3  13.027416        0   
12602  12602  20180919      9    20         3  24.369568        0   
12603  12603  20180919      9    22         3  19.948183        0   
12604  12604  20180919      9    23         3  38.900371        0   

       rushing_hour  max_speed  min_speed  mean_speed  last_speed  next_speed  
0                 0  49.457054  38.352760   44.606355   43.002930   46.118696  
1          

In [247]:
# transfer df to xgb training matrix, Dmatrix
train_full_label = train_df.speed
train_full_features = train_df.drop(['id','date_id','speed'],axis=1)
train_full_DM = xgb.DMatrix(train_full_features,label=train_full_label)

train_s_label = train_s.speed
train_s_features = train_s.drop(['id','date_id','speed'],axis=1)
train_DM = xgb.DMatrix(train_s_features,label=train_s_label)

validation_label = validation.speed
validation_features = validation.drop(['id','date_id','speed'],axis=1)
validation_DM = xgb.DMatrix(validation_features,label=validation_label)

test_df_features = test_df.drop(['id','date_id','sort_id'],axis=1)
test_DM = xgb.DMatrix(test_df_features)

In [248]:
#objective='gbtree', colsample_bytree=0.3, learning_rate=0.1, max_depth=8,alpha = 8, n_estimators=500, reg_lambda=1
params={'booster':'gbtree', #gblinear, gbtree
        'gamma':0.1, #0.1
        'min_child_weight':1.1,#1.1
        'max_depth':8,
        'lambda':10, #l2 10
        'subsample':0.7,
        'colsample_bytree':0.7,
        'colsample_bylevel':0.7,
        'eta': 0.01,
        #'tree_method':'exact',
        'seed':0,
        'nthread':12}
watchlist = [(train_DM,'train'),(validation_DM,'val')]

#trian_and_validation
model = xgb.train(params,train_DM,num_boost_round=3000,evals=watchlist)


[0]	train-rmse:34.67866	val-rmse:34.76448
[1]	train-rmse:34.34900	val-rmse:34.43367
[2]	train-rmse:34.01638	val-rmse:34.10039
[3]	train-rmse:33.68609	val-rmse:33.76693
[4]	train-rmse:33.36793	val-rmse:33.45068
[5]	train-rmse:33.04429	val-rmse:33.12593
[6]	train-rmse:32.72567	val-rmse:32.80797
[7]	train-rmse:32.41345	val-rmse:32.49488
[8]	train-rmse:32.10054	val-rmse:32.18255
[9]	train-rmse:31.78883	val-rmse:31.86807
[10]	train-rmse:31.48333	val-rmse:31.56808
[11]	train-rmse:31.17950	val-rmse:31.26405
[12]	train-rmse:30.87729	val-rmse:30.96002
[13]	train-rmse:30.58389	val-rmse:30.66646
[14]	train-rmse:30.29009	val-rmse:30.37222
[15]	train-rmse:29.99903	val-rmse:30.08467
[16]	train-rmse:29.70859	val-rmse:29.79280
[17]	train-rmse:29.42405	val-rmse:29.51040
[18]	train-rmse:29.14022	val-rmse:29.22611
[19]	train-rmse:28.86295	val-rmse:28.94710
[20]	train-rmse:28.58674	val-rmse:28.67463
[21]	train-rmse:28.31572	val-rmse:28.40475
[22]	train-rmse:28.04264	val-rmse:28.13160
[23]	train-rmse:27.77

[192]	train-rmse:6.37453	val-rmse:6.77480
[193]	train-rmse:6.32963	val-rmse:6.73246
[194]	train-rmse:6.28483	val-rmse:6.69089
[195]	train-rmse:6.24101	val-rmse:6.65082
[196]	train-rmse:6.20129	val-rmse:6.61530
[197]	train-rmse:6.15751	val-rmse:6.57350
[198]	train-rmse:6.11873	val-rmse:6.53947
[199]	train-rmse:6.07626	val-rmse:6.50005
[200]	train-rmse:6.03512	val-rmse:6.46138
[201]	train-rmse:5.99717	val-rmse:6.42812
[202]	train-rmse:5.95620	val-rmse:6.38888
[203]	train-rmse:5.91550	val-rmse:6.34964
[204]	train-rmse:5.87923	val-rmse:6.31739
[205]	train-rmse:5.84398	val-rmse:6.28623
[206]	train-rmse:5.80477	val-rmse:6.25088
[207]	train-rmse:5.76751	val-rmse:6.21642
[208]	train-rmse:5.72938	val-rmse:6.18066
[209]	train-rmse:5.69154	val-rmse:6.14473
[210]	train-rmse:5.65759	val-rmse:6.11443
[211]	train-rmse:5.62439	val-rmse:6.08601
[212]	train-rmse:5.58980	val-rmse:6.05440
[213]	train-rmse:5.55615	val-rmse:6.02508
[214]	train-rmse:5.52475	val-rmse:5.99811
[215]	train-rmse:5.48961	val-rmse:

[388]	train-rmse:3.14360	val-rmse:3.96362
[389]	train-rmse:3.13929	val-rmse:3.95838
[390]	train-rmse:3.13541	val-rmse:3.95421
[391]	train-rmse:3.13262	val-rmse:3.95289
[392]	train-rmse:3.12776	val-rmse:3.94841
[393]	train-rmse:3.12323	val-rmse:3.94452
[394]	train-rmse:3.11852	val-rmse:3.94020
[395]	train-rmse:3.11443	val-rmse:3.93656
[396]	train-rmse:3.11035	val-rmse:3.93293
[397]	train-rmse:3.10751	val-rmse:3.93149
[398]	train-rmse:3.10333	val-rmse:3.92770
[399]	train-rmse:3.09922	val-rmse:3.92363
[400]	train-rmse:3.09530	val-rmse:3.91924
[401]	train-rmse:3.09351	val-rmse:3.91836
[402]	train-rmse:3.08960	val-rmse:3.91526
[403]	train-rmse:3.08730	val-rmse:3.91414
[404]	train-rmse:3.08411	val-rmse:3.91127
[405]	train-rmse:3.08191	val-rmse:3.91018
[406]	train-rmse:3.07836	val-rmse:3.90652
[407]	train-rmse:3.07425	val-rmse:3.90266
[408]	train-rmse:3.07219	val-rmse:3.90172
[409]	train-rmse:3.06900	val-rmse:3.89904
[410]	train-rmse:3.06568	val-rmse:3.89616
[411]	train-rmse:3.06236	val-rmse:

[584]	train-rmse:2.78040	val-rmse:3.68896
[585]	train-rmse:2.77986	val-rmse:3.68849
[586]	train-rmse:2.77919	val-rmse:3.68854
[587]	train-rmse:2.77851	val-rmse:3.68783
[588]	train-rmse:2.77783	val-rmse:3.68750
[589]	train-rmse:2.77705	val-rmse:3.68712
[590]	train-rmse:2.77605	val-rmse:3.68647
[591]	train-rmse:2.77475	val-rmse:3.68537
[592]	train-rmse:2.77420	val-rmse:3.68547
[593]	train-rmse:2.77353	val-rmse:3.68506
[594]	train-rmse:2.77233	val-rmse:3.68424
[595]	train-rmse:2.77135	val-rmse:3.68406
[596]	train-rmse:2.77108	val-rmse:3.68397
[597]	train-rmse:2.76940	val-rmse:3.68222
[598]	train-rmse:2.76874	val-rmse:3.68191
[599]	train-rmse:2.76809	val-rmse:3.68170
[600]	train-rmse:2.76758	val-rmse:3.68152
[601]	train-rmse:2.76655	val-rmse:3.68075
[602]	train-rmse:2.76619	val-rmse:3.68095
[603]	train-rmse:2.76589	val-rmse:3.68094
[604]	train-rmse:2.76574	val-rmse:3.68096
[605]	train-rmse:2.76466	val-rmse:3.67930
[606]	train-rmse:2.76411	val-rmse:3.67925
[607]	train-rmse:2.76288	val-rmse:

[780]	train-rmse:2.64624	val-rmse:3.61941
[781]	train-rmse:2.64554	val-rmse:3.61893
[782]	train-rmse:2.64546	val-rmse:3.61901
[783]	train-rmse:2.64492	val-rmse:3.61838
[784]	train-rmse:2.64456	val-rmse:3.61795
[785]	train-rmse:2.64411	val-rmse:3.61759
[786]	train-rmse:2.64364	val-rmse:3.61708
[787]	train-rmse:2.64335	val-rmse:3.61713
[788]	train-rmse:2.64232	val-rmse:3.61628
[789]	train-rmse:2.64198	val-rmse:3.61634
[790]	train-rmse:2.64128	val-rmse:3.61613
[791]	train-rmse:2.64029	val-rmse:3.61548
[792]	train-rmse:2.64017	val-rmse:3.61538
[793]	train-rmse:2.63965	val-rmse:3.61537
[794]	train-rmse:2.63920	val-rmse:3.61535
[795]	train-rmse:2.63846	val-rmse:3.61523
[796]	train-rmse:2.63736	val-rmse:3.61510
[797]	train-rmse:2.63711	val-rmse:3.61515
[798]	train-rmse:2.63632	val-rmse:3.61469
[799]	train-rmse:2.63599	val-rmse:3.61447
[800]	train-rmse:2.63553	val-rmse:3.61429
[801]	train-rmse:2.63505	val-rmse:3.61402
[802]	train-rmse:2.63458	val-rmse:3.61401
[803]	train-rmse:2.63385	val-rmse:

...

[2123]	train-rmse:2.07834	val-rmse:3.51838
[2124]	train-rmse:2.07820	val-rmse:3.51841
[2125]	train-rmse:2.07778	val-rmse:3.51815
[2126]	train-rmse:2.07766	val-rmse:3.51818
[2127]	train-rmse:2.07716	val-rmse:3.51837
[2128]	train-rmse:2.07659	val-rmse:3.51842
[2129]	train-rmse:2.07634	val-rmse:3.51842
[2130]	train-rmse:2.07609	val-rmse:3.51839
[2131]	train-rmse:2.07555	val-rmse:3.51846
[2132]	train-rmse:2.07510	val-rmse:3.51823
[2133]	train-rmse:2.07505	val-rmse:3.51824
[2134]	train-rmse:2.07461	val-rmse:3.51805
[2135]	train-rmse:2.07435	val-rmse:3.51803
[2136]	train-rmse:2.07382	val-rmse:3.51810
[2137]	train-rmse:2.07358	val-rmse:3.51814
[2138]	train-rmse:2.07317	val-rmse:3.51820
[2139]	train-rmse:2.07293	val-rmse:3.51809
[2140]	train-rmse:2.07258	val-rmse:3.51785
[2141]	train-rmse:2.07229	val-rmse:3.51796
[2142]	train-rmse:2.07179	val-rmse:3.51798
[2143]	train-rmse:2.07161	val-rmse:3.51791
[2144]	train-rmse:2.07148	val-rmse:3.51799
[2145]	train-rmse:2.07100	val-rmse:3.51793
[2146]	trai

[2314]	train-rmse:2.01773	val-rmse:3.51541
[2315]	train-rmse:2.01751	val-rmse:3.51547
[2316]	train-rmse:2.01729	val-rmse:3.51555
[2317]	train-rmse:2.01683	val-rmse:3.51543
[2318]	train-rmse:2.01658	val-rmse:3.51538
[2319]	train-rmse:2.01640	val-rmse:3.51542
[2320]	train-rmse:2.01588	val-rmse:3.51536
[2321]	train-rmse:2.01547	val-rmse:3.51541
[2322]	train-rmse:2.01532	val-rmse:3.51535
[2323]	train-rmse:2.01496	val-rmse:3.51539
[2324]	train-rmse:2.01483	val-rmse:3.51533
[2325]	train-rmse:2.01453	val-rmse:3.51540
[2326]	train-rmse:2.01398	val-rmse:3.51538
[2327]	train-rmse:2.01385	val-rmse:3.51536
[2328]	train-rmse:2.01332	val-rmse:3.51550
[2329]	train-rmse:2.01299	val-rmse:3.51539
[2330]	train-rmse:2.01275	val-rmse:3.51538
[2331]	train-rmse:2.01236	val-rmse:3.51544
[2332]	train-rmse:2.01209	val-rmse:3.51569
[2333]	train-rmse:2.01185	val-rmse:3.51573
[2334]	train-rmse:2.01150	val-rmse:3.51544
[2335]	train-rmse:2.01129	val-rmse:3.51540
[2336]	train-rmse:2.01099	val-rmse:3.51532
[2337]	trai

[2505]	train-rmse:1.95729	val-rmse:3.51183
[2506]	train-rmse:1.95690	val-rmse:3.51161
[2507]	train-rmse:1.95656	val-rmse:3.51158
[2508]	train-rmse:1.95615	val-rmse:3.51136
[2509]	train-rmse:1.95594	val-rmse:3.51132
[2510]	train-rmse:1.95530	val-rmse:3.51108
[2511]	train-rmse:1.95515	val-rmse:3.51099
[2512]	train-rmse:1.95491	val-rmse:3.51099
[2513]	train-rmse:1.95455	val-rmse:3.51094
[2514]	train-rmse:1.95437	val-rmse:3.51093
[2515]	train-rmse:1.95408	val-rmse:3.51100
[2516]	train-rmse:1.95389	val-rmse:3.51106
[2517]	train-rmse:1.95341	val-rmse:3.51119
[2518]	train-rmse:1.95291	val-rmse:3.51120
[2519]	train-rmse:1.95263	val-rmse:3.51096
[2520]	train-rmse:1.95248	val-rmse:3.51091
[2521]	train-rmse:1.95236	val-rmse:3.51100
[2522]	train-rmse:1.95198	val-rmse:3.51079
[2523]	train-rmse:1.95155	val-rmse:3.51067
[2524]	train-rmse:1.95132	val-rmse:3.51082
[2525]	train-rmse:1.95094	val-rmse:3.51083
[2526]	train-rmse:1.95053	val-rmse:3.51092
[2527]	train-rmse:1.95036	val-rmse:3.51094
[2528]	trai

[2696]	train-rmse:1.89772	val-rmse:3.51108
[2697]	train-rmse:1.89735	val-rmse:3.51106
[2698]	train-rmse:1.89708	val-rmse:3.51111
[2699]	train-rmse:1.89672	val-rmse:3.51106
[2700]	train-rmse:1.89651	val-rmse:3.51121
[2701]	train-rmse:1.89606	val-rmse:3.51133
[2702]	train-rmse:1.89559	val-rmse:3.51134
[2703]	train-rmse:1.89511	val-rmse:3.51128
[2704]	train-rmse:1.89480	val-rmse:3.51136
[2705]	train-rmse:1.89459	val-rmse:3.51132
[2706]	train-rmse:1.89402	val-rmse:3.51133
[2707]	train-rmse:1.89389	val-rmse:3.51132
[2708]	train-rmse:1.89369	val-rmse:3.51128
[2709]	train-rmse:1.89355	val-rmse:3.51125
[2710]	train-rmse:1.89336	val-rmse:3.51133
[2711]	train-rmse:1.89298	val-rmse:3.51126
[2712]	train-rmse:1.89282	val-rmse:3.51118
[2713]	train-rmse:1.89255	val-rmse:3.51112
[2714]	train-rmse:1.89236	val-rmse:3.51109
[2715]	train-rmse:1.89202	val-rmse:3.51112
[2716]	train-rmse:1.89187	val-rmse:3.51094
[2717]	train-rmse:1.89162	val-rmse:3.51096
[2718]	train-rmse:1.89125	val-rmse:3.51098
[2719]	trai

[2887]	train-rmse:1.84032	val-rmse:3.51207
[2888]	train-rmse:1.83993	val-rmse:3.51200
[2889]	train-rmse:1.83963	val-rmse:3.51213
[2890]	train-rmse:1.83920	val-rmse:3.51228
[2891]	train-rmse:1.83909	val-rmse:3.51233
[2892]	train-rmse:1.83873	val-rmse:3.51247
[2893]	train-rmse:1.83848	val-rmse:3.51256
[2894]	train-rmse:1.83836	val-rmse:3.51257
[2895]	train-rmse:1.83801	val-rmse:3.51271
[2896]	train-rmse:1.83769	val-rmse:3.51258
[2897]	train-rmse:1.83733	val-rmse:3.51272
[2898]	train-rmse:1.83711	val-rmse:3.51255
[2899]	train-rmse:1.83675	val-rmse:3.51237
[2900]	train-rmse:1.83649	val-rmse:3.51235
[2901]	train-rmse:1.83621	val-rmse:3.51236
[2902]	train-rmse:1.83581	val-rmse:3.51238
[2903]	train-rmse:1.83555	val-rmse:3.51265
[2904]	train-rmse:1.83527	val-rmse:3.51250
[2905]	train-rmse:1.83496	val-rmse:3.51245
[2906]	train-rmse:1.83472	val-rmse:3.51248
[2907]	train-rmse:1.83448	val-rmse:3.51237
[2908]	train-rmse:1.83428	val-rmse:3.51242
[2909]	train-rmse:1.83395	val-rmse:3.51242
[2910]	trai

In [249]:
#all train data going to model training, no validation
watchlist_full = [(train_DM,'train')]
model_full = xgb.train(params,train_full_DM,num_boost_round=3000,evals=watchlist_full)

[0]	train-rmse:34.67921
[1]	train-rmse:34.34574
[2]	train-rmse:34.01415
[3]	train-rmse:33.68401
[4]	train-rmse:33.35744
[5]	train-rmse:33.03354
[6]	train-rmse:32.71574
[7]	train-rmse:32.40013
[8]	train-rmse:32.08728
[9]	train-rmse:31.77648
[10]	train-rmse:31.47709
[11]	train-rmse:31.17201
[12]	train-rmse:30.87251
[13]	train-rmse:30.57392
[14]	train-rmse:30.27896
[15]	train-rmse:29.98811
[16]	train-rmse:29.69803
[17]	train-rmse:29.41153
[18]	train-rmse:29.12944
[19]	train-rmse:28.85366
[20]	train-rmse:28.58117
[21]	train-rmse:28.31196
[22]	train-rmse:28.03836
[23]	train-rmse:27.76753
[24]	train-rmse:27.50347
[25]	train-rmse:27.23863
[26]	train-rmse:26.97677
[27]	train-rmse:26.71915
[28]	train-rmse:26.46435
[29]	train-rmse:26.21435
[30]	train-rmse:25.96173
[31]	train-rmse:25.71186
[32]	train-rmse:25.46475
[33]	train-rmse:25.22121
[34]	train-rmse:24.97975
[35]	train-rmse:24.74015
[36]	train-rmse:24.50235
[37]	train-rmse:24.26852
[38]	train-rmse:24.03860
[39]	train-rmse:23.81063
[40]	train

[327]	train-rmse:3.45509
[328]	train-rmse:3.44829
[329]	train-rmse:3.44036
[330]	train-rmse:3.43262
[331]	train-rmse:3.42511
[332]	train-rmse:3.41764
[333]	train-rmse:3.40980
[334]	train-rmse:3.40180
[335]	train-rmse:3.39416
[336]	train-rmse:3.38671
[337]	train-rmse:3.38068
[338]	train-rmse:3.37314
[339]	train-rmse:3.36578
[340]	train-rmse:3.35829
[341]	train-rmse:3.35221
[342]	train-rmse:3.34482
[343]	train-rmse:3.33937
[344]	train-rmse:3.33441
[345]	train-rmse:3.32822
[346]	train-rmse:3.32222
[347]	train-rmse:3.31537
[348]	train-rmse:3.30945
[349]	train-rmse:3.30248
[350]	train-rmse:3.29794
[351]	train-rmse:3.29171
[352]	train-rmse:3.28749
[353]	train-rmse:3.28200
[354]	train-rmse:3.27584
[355]	train-rmse:3.26981
[356]	train-rmse:3.26397
[357]	train-rmse:3.25978
[358]	train-rmse:3.25410
[359]	train-rmse:3.24870
[360]	train-rmse:3.24462
[361]	train-rmse:3.24014
[362]	train-rmse:3.23439
[363]	train-rmse:3.23043
[364]	train-rmse:3.22721
[365]	train-rmse:3.22192
[366]	train-rmse:3.21820


[655]	train-rmse:2.73184
[656]	train-rmse:2.73168
[657]	train-rmse:2.73114
[658]	train-rmse:2.73000
[659]	train-rmse:2.72896
[660]	train-rmse:2.72867
[661]	train-rmse:2.72813
[662]	train-rmse:2.72765
[663]	train-rmse:2.72727
[664]	train-rmse:2.72708
[665]	train-rmse:2.72643
[666]	train-rmse:2.72529
[667]	train-rmse:2.72474
[668]	train-rmse:2.72425
[669]	train-rmse:2.72332
[670]	train-rmse:2.72308
[671]	train-rmse:2.72282
[672]	train-rmse:2.72215
[673]	train-rmse:2.72077
[674]	train-rmse:2.72054
[675]	train-rmse:2.72031
[676]	train-rmse:2.71952
[677]	train-rmse:2.71860
[678]	train-rmse:2.71769
[679]	train-rmse:2.71729
[680]	train-rmse:2.71710
[681]	train-rmse:2.71647
[682]	train-rmse:2.71585
[683]	train-rmse:2.71571
[684]	train-rmse:2.71534
[685]	train-rmse:2.71487
[686]	train-rmse:2.71466
[687]	train-rmse:2.71414
[688]	train-rmse:2.71396
[689]	train-rmse:2.71376
[690]	train-rmse:2.71260
[691]	train-rmse:2.71168
[692]	train-rmse:2.71097
[693]	train-rmse:2.71036
[694]	train-rmse:2.70956


[983]	train-rmse:2.56214
[984]	train-rmse:2.56171
[985]	train-rmse:2.56145
[986]	train-rmse:2.56132
[987]	train-rmse:2.56067
[988]	train-rmse:2.56052
[989]	train-rmse:2.55975
[990]	train-rmse:2.55907
[991]	train-rmse:2.55873
[992]	train-rmse:2.55821
[993]	train-rmse:2.55794
[994]	train-rmse:2.55763
[995]	train-rmse:2.55727
[996]	train-rmse:2.55695
[997]	train-rmse:2.55683
[998]	train-rmse:2.55631
[999]	train-rmse:2.55564
[1000]	train-rmse:2.55535
[1001]	train-rmse:2.55488
[1002]	train-rmse:2.55430
[1003]	train-rmse:2.55396
[1004]	train-rmse:2.55330
[1005]	train-rmse:2.55275
[1006]	train-rmse:2.55232
[1007]	train-rmse:2.55219
[1008]	train-rmse:2.55178
[1009]	train-rmse:2.55126
[1010]	train-rmse:2.55096
[1011]	train-rmse:2.55076
[1012]	train-rmse:2.55024
[1013]	train-rmse:2.55018
[1014]	train-rmse:2.54977
[1015]	train-rmse:2.54908
[1016]	train-rmse:2.54881
[1017]	train-rmse:2.54834
[1018]	train-rmse:2.54783
[1019]	train-rmse:2.54725
[1020]	train-rmse:2.54654
[1021]	train-rmse:2.54615
[10

[1299]	train-rmse:2.42590
[1300]	train-rmse:2.42561
[1301]	train-rmse:2.42531
[1302]	train-rmse:2.42499
[1303]	train-rmse:2.42462
[1304]	train-rmse:2.42414
[1305]	train-rmse:2.42356
[1306]	train-rmse:2.42288
[1307]	train-rmse:2.42246
[1308]	train-rmse:2.42221
[1309]	train-rmse:2.42172
[1310]	train-rmse:2.42101
[1311]	train-rmse:2.42075
[1312]	train-rmse:2.42031
[1313]	train-rmse:2.41992
[1314]	train-rmse:2.41966
[1315]	train-rmse:2.41936
[1316]	train-rmse:2.41868
[1317]	train-rmse:2.41827
[1318]	train-rmse:2.41805
[1319]	train-rmse:2.41766
[1320]	train-rmse:2.41712
[1321]	train-rmse:2.41670
[1322]	train-rmse:2.41649
[1323]	train-rmse:2.41601
[1324]	train-rmse:2.41585
[1325]	train-rmse:2.41527
[1326]	train-rmse:2.41501
[1327]	train-rmse:2.41438
[1328]	train-rmse:2.41412
[1329]	train-rmse:2.41368
[1330]	train-rmse:2.41331
[1331]	train-rmse:2.41307
[1332]	train-rmse:2.41249
[1333]	train-rmse:2.41202
[1334]	train-rmse:2.41142
[1335]	train-rmse:2.41094
[1336]	train-rmse:2.41048
[1337]	train

[1615]	train-rmse:2.30238
[1616]	train-rmse:2.30210
[1617]	train-rmse:2.30176
[1618]	train-rmse:2.30144
[1619]	train-rmse:2.30120
[1620]	train-rmse:2.30084
[1621]	train-rmse:2.30031
[1622]	train-rmse:2.29996
[1623]	train-rmse:2.29932
[1624]	train-rmse:2.29872
[1625]	train-rmse:2.29834
[1626]	train-rmse:2.29783
[1627]	train-rmse:2.29777
[1628]	train-rmse:2.29763
[1629]	train-rmse:2.29743
[1630]	train-rmse:2.29723
[1631]	train-rmse:2.29670
[1632]	train-rmse:2.29660
[1633]	train-rmse:2.29623
[1634]	train-rmse:2.29549
[1635]	train-rmse:2.29517
[1636]	train-rmse:2.29485
[1637]	train-rmse:2.29457
[1638]	train-rmse:2.29441
[1639]	train-rmse:2.29378
[1640]	train-rmse:2.29346
[1641]	train-rmse:2.29318
[1642]	train-rmse:2.29267
[1643]	train-rmse:2.29227
[1644]	train-rmse:2.29212
[1645]	train-rmse:2.29137
[1646]	train-rmse:2.29102
[1647]	train-rmse:2.29061
[1648]	train-rmse:2.29046
[1649]	train-rmse:2.29009
[1650]	train-rmse:2.28983
[1651]	train-rmse:2.28950
[1652]	train-rmse:2.28926
[1653]	train

[1931]	train-rmse:2.19019
[1932]	train-rmse:2.18980
[1933]	train-rmse:2.18952
[1934]	train-rmse:2.18929
[1935]	train-rmse:2.18903
[1936]	train-rmse:2.18874
[1937]	train-rmse:2.18851
[1938]	train-rmse:2.18819
[1939]	train-rmse:2.18783
[1940]	train-rmse:2.18741
[1941]	train-rmse:2.18712
[1942]	train-rmse:2.18648
[1943]	train-rmse:2.18631
[1944]	train-rmse:2.18577
[1945]	train-rmse:2.18529
[1946]	train-rmse:2.18468
[1947]	train-rmse:2.18441
[1948]	train-rmse:2.18414
[1949]	train-rmse:2.18392
[1950]	train-rmse:2.18356
[1951]	train-rmse:2.18325
[1952]	train-rmse:2.18286
[1953]	train-rmse:2.18232
[1954]	train-rmse:2.18195
[1955]	train-rmse:2.18180
[1956]	train-rmse:2.18158
[1957]	train-rmse:2.18149
[1958]	train-rmse:2.18111
[1959]	train-rmse:2.18088
[1960]	train-rmse:2.18074
[1961]	train-rmse:2.18041
[1962]	train-rmse:2.17997
[1963]	train-rmse:2.17944
[1964]	train-rmse:2.17913
[1965]	train-rmse:2.17873
[1966]	train-rmse:2.17858
[1967]	train-rmse:2.17827
[1968]	train-rmse:2.17800
[1969]	train

[2247]	train-rmse:2.08986
[2248]	train-rmse:2.08958
[2249]	train-rmse:2.08929
[2250]	train-rmse:2.08886
[2251]	train-rmse:2.08869
[2252]	train-rmse:2.08835
[2253]	train-rmse:2.08819
[2254]	train-rmse:2.08793
[2255]	train-rmse:2.08766
[2256]	train-rmse:2.08722
[2257]	train-rmse:2.08703
[2258]	train-rmse:2.08668
[2259]	train-rmse:2.08643
[2260]	train-rmse:2.08614
[2261]	train-rmse:2.08587
[2262]	train-rmse:2.08574
[2263]	train-rmse:2.08538
[2264]	train-rmse:2.08513
[2265]	train-rmse:2.08468
[2266]	train-rmse:2.08436
[2267]	train-rmse:2.08397
[2268]	train-rmse:2.08360
[2269]	train-rmse:2.08334
[2270]	train-rmse:2.08291
[2271]	train-rmse:2.08261
[2272]	train-rmse:2.08213
[2273]	train-rmse:2.08168
[2274]	train-rmse:2.08148
[2275]	train-rmse:2.08091
[2276]	train-rmse:2.08072
[2277]	train-rmse:2.08034
[2278]	train-rmse:2.07996
[2279]	train-rmse:2.07959
[2280]	train-rmse:2.07921
[2281]	train-rmse:2.07875
[2282]	train-rmse:2.07840
[2283]	train-rmse:2.07803
[2284]	train-rmse:2.07782
[2285]	train

[2563]	train-rmse:1.99022
[2564]	train-rmse:1.99003
[2565]	train-rmse:1.98969
[2566]	train-rmse:1.98934
[2567]	train-rmse:1.98921
[2568]	train-rmse:1.98901
[2569]	train-rmse:1.98881
[2570]	train-rmse:1.98847
[2571]	train-rmse:1.98823
[2572]	train-rmse:1.98792
[2573]	train-rmse:1.98759
[2574]	train-rmse:1.98739
[2575]	train-rmse:1.98707
[2576]	train-rmse:1.98673
[2577]	train-rmse:1.98634
[2578]	train-rmse:1.98615
[2579]	train-rmse:1.98597
[2580]	train-rmse:1.98554
[2581]	train-rmse:1.98523
[2582]	train-rmse:1.98503
[2583]	train-rmse:1.98456
[2584]	train-rmse:1.98442
[2585]	train-rmse:1.98412
[2586]	train-rmse:1.98381
[2587]	train-rmse:1.98361
[2588]	train-rmse:1.98333
[2589]	train-rmse:1.98298
[2590]	train-rmse:1.98265
[2591]	train-rmse:1.98233
[2592]	train-rmse:1.98212
[2593]	train-rmse:1.98209
[2594]	train-rmse:1.98170
[2595]	train-rmse:1.98150
[2596]	train-rmse:1.98132
[2597]	train-rmse:1.98083
[2598]	train-rmse:1.98052
[2599]	train-rmse:1.98006
[2600]	train-rmse:1.97983
[2601]	train

[2879]	train-rmse:1.90206
[2880]	train-rmse:1.90161
[2881]	train-rmse:1.90148
[2882]	train-rmse:1.90131
[2883]	train-rmse:1.90105
[2884]	train-rmse:1.90084
[2885]	train-rmse:1.90061
[2886]	train-rmse:1.90020
[2887]	train-rmse:1.89986
[2888]	train-rmse:1.89950
[2889]	train-rmse:1.89908
[2890]	train-rmse:1.89872
[2891]	train-rmse:1.89830
[2892]	train-rmse:1.89775
[2893]	train-rmse:1.89760
[2894]	train-rmse:1.89725
[2895]	train-rmse:1.89681
[2896]	train-rmse:1.89657
[2897]	train-rmse:1.89642
[2898]	train-rmse:1.89615
[2899]	train-rmse:1.89586
[2900]	train-rmse:1.89560
[2901]	train-rmse:1.89523
[2902]	train-rmse:1.89495
[2903]	train-rmse:1.89478
[2904]	train-rmse:1.89435
[2905]	train-rmse:1.89407
[2906]	train-rmse:1.89369
[2907]	train-rmse:1.89336
[2908]	train-rmse:1.89308
[2909]	train-rmse:1.89265
[2910]	train-rmse:1.89244
[2911]	train-rmse:1.89223
[2912]	train-rmse:1.89199
[2913]	train-rmse:1.89170
[2914]	train-rmse:1.89150
[2915]	train-rmse:1.89139
[2916]	train-rmse:1.89100
[2917]	train

In [250]:
test_preds = test_df.loc[:,['id']]
test_preds['speed'] = model.predict(test_DM)
test_preds_full = test_df.loc[:,['id']]
test_preds_full['speed'] = model_full.predict(test_DM)
print(test_preds)
#two verion, one is with validation, one is full list.
np.savetxt('prediction_xgbversion15_closespeed_val.csv', test_preds,fmt="%d,%.6f" ,header='id,speed', delimiter=',', comments='')
np.savetxt('prediction_xgbversion15_closespeed_ful.csv', test_preds_full,fmt="%d,%.6f" ,header='id,speed', delimiter=',', comments='')

        id      speed
0        0  48.180996
1        1  47.477943
2        2  40.680580
3        3  32.469860
4        4  39.824718
...    ...        ...
3499  3499  13.608505
3500  3500  25.687960
3501  3501  32.565292
3502  3502  26.931799
3503  3503  28.446957

[3504 rows x 2 columns]


In [203]:
#this session is not used!!!!!!!!!!!!!!!!

#Each round update next and last speed after prediction to make model converge 
#but it does not perform well, not used

train_df['str_hour']=train_df['hour'].apply(hours_to_string)
train_df['sort_id']=train_df['date_id']+train_df['str_hour']
train_df = train_df.drop(['str_hour'],axis=1)


for i in range(10):
    #predict test speed
    model = xgb.train(params,train_DM,num_boost_round=7000,evals=watchlist)
    #print(test_df.columns)
    #print(train_df.columns)
    #print(train_s_features.columns)
    #print(test_preds.columns)
    test_preds = test_df.loc[:,['id']]
    
    #print(train_df.columns)
    #print(test_df.columns)
    #print(test_preds.columns)
    test_preds['speed'] = model.predict(test_DM)
    test_df.loc[:,'speed']=test_preds.loc[:,'speed']
    #print(test_preds)
    #print(test_df)
    #update next and last speed to make model converge
        #step 1, take speed from predictions
    temp_test_df = test_df.copy()
    temp_test_df = temp_test_df[['sort_id','speed']]
    #print(temp_test_df)
    temp_df = pd.concat([speed_df,temp_test_df], ignore_index=True) 
    temp_df = temp_df.sort_values('sort_id',ascending=True, ignore_index=True)
        #step 2, take speed from train and combine them
    temp_df['last_speed'] = 0
    temp_df['next_speed'] = 0
    #print(temp_df)
    get_last_speed_for_train(temp_df)
    get_next_speed_for_train(temp_df)
    #print(temp_df)

    transfer_close_speed_to_test(temp_df, test_df)
    transfer_close_speed_to_test(temp_df, train_df)
    #print(train_df.columns)
    #print(test_df)
    
    #row = test_df.loc[0].values.tolist()
    #print(row)

    
    
    train_s, validation = np.split(train_df, [int(.9*len(train_df))])
    

    train_s_label = train_s.speed
    train_s_features = train_s.drop(['id','date_id','speed','sort_id'],axis=1)
    train_DM = xgb.DMatrix(train_s_features,label=train_s_label)

    validation_label = validation.speed
    validation_features = validation.drop(['id','date_id','speed','sort_id'],axis=1)
    validation_DM = xgb.DMatrix(validation_features,label=validation_label)
    test_df_features = test_df.drop(['id','date_id','sort_id','speed'],axis=1)
    test_DM = xgb.DMatrix(test_df_features)
    test_df = test_df.drop(['speed','last_speed','next_speed'],axis=1)



[0]	train-rmse:34.66798	val-rmse:34.75059
[1]	train-rmse:34.33104	val-rmse:34.41071
[2]	train-rmse:33.99761	val-rmse:34.07549
[3]	train-rmse:33.66721	val-rmse:33.74374
[4]	train-rmse:33.34049	val-rmse:33.41588
[5]	train-rmse:33.01697	val-rmse:33.09139
[6]	train-rmse:32.69663	val-rmse:32.77088
[7]	train-rmse:32.37949	val-rmse:32.45169
[8]	train-rmse:32.06539	val-rmse:32.13791
[9]	train-rmse:31.75465	val-rmse:31.82523
[10]	train-rmse:31.44724	val-rmse:31.51625
[11]	train-rmse:31.14276	val-rmse:31.21150
[12]	train-rmse:30.84113	val-rmse:30.90823
[13]	train-rmse:30.54272	val-rmse:30.60886
[14]	train-rmse:30.24771	val-rmse:30.31288
[15]	train-rmse:29.95553	val-rmse:30.02146
[16]	train-rmse:29.66595	val-rmse:29.73139
[17]	train-rmse:29.37968	val-rmse:29.44485
[18]	train-rmse:29.09583	val-rmse:29.16095
[19]	train-rmse:28.81525	val-rmse:28.88164
[20]	train-rmse:28.53703	val-rmse:28.60286
[21]	train-rmse:28.26127	val-rmse:28.32736
[22]	train-rmse:27.98929	val-rmse:28.05649
[23]	train-rmse:27.71

[192]	train-rmse:6.38828	val-rmse:6.91110
[193]	train-rmse:6.34517	val-rmse:6.87075
[194]	train-rmse:6.30286	val-rmse:6.83072
[195]	train-rmse:6.26122	val-rmse:6.79255
[196]	train-rmse:6.22025	val-rmse:6.75607
[197]	train-rmse:6.17866	val-rmse:6.71908
[198]	train-rmse:6.13832	val-rmse:6.68203
[199]	train-rmse:6.09835	val-rmse:6.64498
[200]	train-rmse:6.05949	val-rmse:6.61040
[201]	train-rmse:6.02038	val-rmse:6.57437
[202]	train-rmse:5.98259	val-rmse:6.53910
[203]	train-rmse:5.94489	val-rmse:6.50440
[204]	train-rmse:5.90778	val-rmse:6.47078
[205]	train-rmse:5.86997	val-rmse:6.43531
[206]	train-rmse:5.83325	val-rmse:6.40135
[207]	train-rmse:5.79708	val-rmse:6.36787
[208]	train-rmse:5.76144	val-rmse:6.33590
[209]	train-rmse:5.72631	val-rmse:6.30466
[210]	train-rmse:5.69170	val-rmse:6.27419
[211]	train-rmse:5.65736	val-rmse:6.24209
[212]	train-rmse:5.62341	val-rmse:6.21024
[213]	train-rmse:5.59024	val-rmse:6.17992
[214]	train-rmse:5.55729	val-rmse:6.15077
[215]	train-rmse:5.52460	val-rmse:

[388]	train-rmse:3.39571	val-rmse:4.20424
[389]	train-rmse:3.39282	val-rmse:4.20141
[390]	train-rmse:3.38983	val-rmse:4.19870
[391]	train-rmse:3.38738	val-rmse:4.19683
[392]	train-rmse:3.38464	val-rmse:4.19368
[393]	train-rmse:3.38155	val-rmse:4.19072
[394]	train-rmse:3.37930	val-rmse:4.18878
[395]	train-rmse:3.37698	val-rmse:4.18680
[396]	train-rmse:3.37483	val-rmse:4.18499
[397]	train-rmse:3.37173	val-rmse:4.18196
[398]	train-rmse:3.36926	val-rmse:4.17989
[399]	train-rmse:3.36639	val-rmse:4.17758
[400]	train-rmse:3.36400	val-rmse:4.17583
[401]	train-rmse:3.36185	val-rmse:4.17415
[402]	train-rmse:3.35892	val-rmse:4.17165
[403]	train-rmse:3.35701	val-rmse:4.17009
[404]	train-rmse:3.35485	val-rmse:4.16857
[405]	train-rmse:3.35262	val-rmse:4.16662
[406]	train-rmse:3.35043	val-rmse:4.16428
[407]	train-rmse:3.34770	val-rmse:4.16144
[408]	train-rmse:3.34560	val-rmse:4.15963
[409]	train-rmse:3.34388	val-rmse:4.15846
[410]	train-rmse:3.34076	val-rmse:4.15463
[411]	train-rmse:3.33833	val-rmse:

[584]	train-rmse:3.14113	val-rmse:3.96575
[585]	train-rmse:3.14061	val-rmse:3.96532
[586]	train-rmse:3.13983	val-rmse:3.96408
[587]	train-rmse:3.13939	val-rmse:3.96352
[588]	train-rmse:3.13855	val-rmse:3.96288
[589]	train-rmse:3.13821	val-rmse:3.96253
[590]	train-rmse:3.13735	val-rmse:3.96122
[591]	train-rmse:3.13670	val-rmse:3.96085
[592]	train-rmse:3.13632	val-rmse:3.96054
[593]	train-rmse:3.13597	val-rmse:3.96000
[594]	train-rmse:3.13516	val-rmse:3.95926
[595]	train-rmse:3.13453	val-rmse:3.95829
[596]	train-rmse:3.13396	val-rmse:3.95768
[597]	train-rmse:3.13363	val-rmse:3.95726
[598]	train-rmse:3.13330	val-rmse:3.95683
[599]	train-rmse:3.13264	val-rmse:3.95598
[600]	train-rmse:3.13119	val-rmse:3.95437
[601]	train-rmse:3.13065	val-rmse:3.95394
[602]	train-rmse:3.13001	val-rmse:3.95317
[603]	train-rmse:3.12958	val-rmse:3.95255
[604]	train-rmse:3.12883	val-rmse:3.95199
[605]	train-rmse:3.12827	val-rmse:3.95137
[606]	train-rmse:3.12759	val-rmse:3.95099
[607]	train-rmse:3.12727	val-rmse:

[780]	train-rmse:3.04475	val-rmse:3.87898
[781]	train-rmse:3.04438	val-rmse:3.87866
[782]	train-rmse:3.04407	val-rmse:3.87861
[783]	train-rmse:3.04308	val-rmse:3.87748
[784]	train-rmse:3.04238	val-rmse:3.87666
[785]	train-rmse:3.04160	val-rmse:3.87582
[786]	train-rmse:3.04126	val-rmse:3.87549
[787]	train-rmse:3.04018	val-rmse:3.87434
[788]	train-rmse:3.03992	val-rmse:3.87409
[789]	train-rmse:3.03931	val-rmse:3.87345
[790]	train-rmse:3.03910	val-rmse:3.87332
[791]	train-rmse:3.03870	val-rmse:3.87282
[792]	train-rmse:3.03864	val-rmse:3.87280
[793]	train-rmse:3.03776	val-rmse:3.87230
[794]	train-rmse:3.03762	val-rmse:3.87237
[795]	train-rmse:3.03738	val-rmse:3.87237
[796]	train-rmse:3.03643	val-rmse:3.87102
[797]	train-rmse:3.03628	val-rmse:3.87105
[798]	train-rmse:3.03584	val-rmse:3.87078
[799]	train-rmse:3.03570	val-rmse:3.87065
[800]	train-rmse:3.03507	val-rmse:3.87013
[801]	train-rmse:3.03461	val-rmse:3.86979
[802]	train-rmse:3.03373	val-rmse:3.86896
[803]	train-rmse:3.03353	val-rmse:

...

[6277]	train-rmse:3.01503	val-rmse:5.18326
[6278]	train-rmse:3.01498	val-rmse:5.18329
[6279]	train-rmse:3.01492	val-rmse:5.18333
[6280]	train-rmse:3.01489	val-rmse:5.18336
[6281]	train-rmse:3.01481	val-rmse:5.18336
[6282]	train-rmse:3.01472	val-rmse:5.18338
[6283]	train-rmse:3.01464	val-rmse:5.18341
[6284]	train-rmse:3.01457	val-rmse:5.18340
[6285]	train-rmse:3.01452	val-rmse:5.18336
[6286]	train-rmse:3.01448	val-rmse:5.18338
[6287]	train-rmse:3.01440	val-rmse:5.18339
[6288]	train-rmse:3.01431	val-rmse:5.18353
[6289]	train-rmse:3.01425	val-rmse:5.18347
[6290]	train-rmse:3.01419	val-rmse:5.18342
[6291]	train-rmse:3.01414	val-rmse:5.18337
[6292]	train-rmse:3.01406	val-rmse:5.18334
[6293]	train-rmse:3.01404	val-rmse:5.18337
[6294]	train-rmse:3.01399	val-rmse:5.18350
[6295]	train-rmse:3.01395	val-rmse:5.18360
[6296]	train-rmse:3.01390	val-rmse:5.18359
[6297]	train-rmse:3.01387	val-rmse:5.18365
[6298]	train-rmse:3.01380	val-rmse:5.18377
[6299]	train-rmse:3.01375	val-rmse:5.18382
[6300]	trai

[6468]	train-rmse:3.00255	val-rmse:5.18234
[6469]	train-rmse:3.00252	val-rmse:5.18235
[6470]	train-rmse:3.00241	val-rmse:5.18225
[6471]	train-rmse:3.00234	val-rmse:5.18224
[6472]	train-rmse:3.00232	val-rmse:5.18222
[6473]	train-rmse:3.00226	val-rmse:5.18220
[6474]	train-rmse:3.00221	val-rmse:5.18219
[6475]	train-rmse:3.00215	val-rmse:5.18216
[6476]	train-rmse:3.00207	val-rmse:5.18220
[6477]	train-rmse:3.00199	val-rmse:5.18210
[6478]	train-rmse:3.00192	val-rmse:5.18210
[6479]	train-rmse:3.00188	val-rmse:5.18207
[6480]	train-rmse:3.00182	val-rmse:5.18208
[6481]	train-rmse:3.00175	val-rmse:5.18201
[6482]	train-rmse:3.00171	val-rmse:5.18196
[6483]	train-rmse:3.00161	val-rmse:5.18204
[6484]	train-rmse:3.00155	val-rmse:5.18190
[6485]	train-rmse:3.00147	val-rmse:5.18192
[6486]	train-rmse:3.00141	val-rmse:5.18188
[6487]	train-rmse:3.00130	val-rmse:5.18185
[6488]	train-rmse:3.00119	val-rmse:5.18183
[6489]	train-rmse:3.00114	val-rmse:5.18189
[6490]	train-rmse:3.00112	val-rmse:5.18200
[6491]	trai

[6659]	train-rmse:2.99032	val-rmse:5.17907
[6660]	train-rmse:2.99020	val-rmse:5.17898
[6661]	train-rmse:2.99017	val-rmse:5.17892
[6662]	train-rmse:2.99013	val-rmse:5.17898
[6663]	train-rmse:2.99003	val-rmse:5.17873
[6664]	train-rmse:2.99001	val-rmse:5.17876
[6665]	train-rmse:2.98997	val-rmse:5.17875
[6666]	train-rmse:2.98991	val-rmse:5.17876
[6667]	train-rmse:2.98984	val-rmse:5.17872
[6668]	train-rmse:2.98973	val-rmse:5.17866
[6669]	train-rmse:2.98964	val-rmse:5.17870
[6670]	train-rmse:2.98959	val-rmse:5.17874
[6671]	train-rmse:2.98954	val-rmse:5.17890
[6672]	train-rmse:2.98942	val-rmse:5.17891
[6673]	train-rmse:2.98932	val-rmse:5.17887
[6674]	train-rmse:2.98922	val-rmse:5.17887
[6675]	train-rmse:2.98918	val-rmse:5.17886
[6676]	train-rmse:2.98911	val-rmse:5.17880
[6677]	train-rmse:2.98904	val-rmse:5.17879
[6678]	train-rmse:2.98897	val-rmse:5.17872
[6679]	train-rmse:2.98894	val-rmse:5.17873
[6680]	train-rmse:2.98884	val-rmse:5.17874
[6681]	train-rmse:2.98881	val-rmse:5.17874
[6682]	trai

[6850]	train-rmse:2.97739	val-rmse:5.17531
[6851]	train-rmse:2.97736	val-rmse:5.17530
[6852]	train-rmse:2.97727	val-rmse:5.17530
[6853]	train-rmse:2.97722	val-rmse:5.17531
[6854]	train-rmse:2.97716	val-rmse:5.17527
[6855]	train-rmse:2.97712	val-rmse:5.17529
[6856]	train-rmse:2.97704	val-rmse:5.17531
[6857]	train-rmse:2.97698	val-rmse:5.17537
[6858]	train-rmse:2.97692	val-rmse:5.17534
[6859]	train-rmse:2.97673	val-rmse:5.17507
[6860]	train-rmse:2.97659	val-rmse:5.17495
[6861]	train-rmse:2.97650	val-rmse:5.17486
[6862]	train-rmse:2.97644	val-rmse:5.17469
[6863]	train-rmse:2.97637	val-rmse:5.17470
[6864]	train-rmse:2.97629	val-rmse:5.17468
[6865]	train-rmse:2.97628	val-rmse:5.17472
[6866]	train-rmse:2.97623	val-rmse:5.17475
[6867]	train-rmse:2.97616	val-rmse:5.17472
[6868]	train-rmse:2.97607	val-rmse:5.17472
[6869]	train-rmse:2.97603	val-rmse:5.17468
[6870]	train-rmse:2.97598	val-rmse:5.17469
[6871]	train-rmse:2.97595	val-rmse:5.17463
[6872]	train-rmse:2.97586	val-rmse:5.17471
[6873]	trai

KeyboardInterrupt: 

In [66]:
#this session is not used!!!!!!!!!!!!!!!!
test_preds = test_df.loc[:,['id']]
test_preds['speed'] = model.predict(test_DM)
test_preds_full = test_df.loc[:,['id']]
test_preds_full['speed'] = model_full.predict(test_DM)
print(test_preds)
np.savetxt('prediction_xgbversion14_closespeed_val.csv', test_preds,fmt="%d,%.6f" ,header='id,speed', delimiter=',', comments='')
np.savetxt('prediction_xgbversion14_closespeed_ful.csv', test_preds_full,fmt="%d,%.6f" ,header='id,speed', delimiter=',', comments='')

        id      speed
0        0  48.276760
1        1  47.409336
2        2  40.311253
3        3  32.338905
4        4  38.907387
...    ...        ...
3499  3499  13.685380
3500  3500  25.478352
3501  3501  28.243845
3502  3502  22.073828
3503  3503  22.110878

[3504 rows x 2 columns]


C:\Users\Administrator\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Administrator\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
